# Connecting to the Spotify API with Spotipy


--------

In [3]:
import sys
import numpy as np
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials

### Create a spotipy manager for Client Credentials

Input your client_id and client_secret as functions in the SpotifyClientCredentials call below, or set environment variables. (see docs below)

<em>Note from the docs:

To support the Authorization Code Flow Spotipy provides a utility method util.prompt_for_user_token that will attempt to authorize the user. You can pass your app credentials directly into the method as arguments, or if you are reluctant to immortalize your app credentials in your source code, you can set environment variables like so:

export SPOTIPY_CLIENT_ID='your-spotify-client-id'
export SPOTIPY_CLIENT_SECRET='your-spotify-client-secret'
export SPOTIPY_REDIRECT_URI='your-app-redirect-url'</em>

In [4]:
client_credentials_manager = SpotifyClientCredentials(client_id='', client_secret='')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

### Test out an API call

In [5]:
sp.user_playlists('spotify', limit=20, offset=0)

{'href': 'https://api.spotify.com/v1/users/spotify/playlists?offset=0&limit=20',
 'items': [{'collaborative': False,
   'external_urls': {'spotify': 'http://open.spotify.com/user/spotify/playlist/46HbECfImnmmxeQP09Ves4'},
   'href': 'https://api.spotify.com/v1/users/spotify/playlists/46HbECfImnmmxeQP09Ves4',
   'id': '46HbECfImnmmxeQP09Ves4',
   'images': [],
   'name': 'This Is:  Johnny Cash',
   'owner': {'external_urls': {'spotify': 'http://open.spotify.com/user/spotify'},
    'href': 'https://api.spotify.com/v1/users/spotify',
    'id': 'spotify',
    'type': 'user',
    'uri': 'spotify:user:spotify'},
   'public': True,
   'snapshot_id': 'ywxjdARDGiRKcxrbnyG7FT7li1zjZURKYbcDuz9HaObqx3qRLlwX4mO6tN9CCYbM',
   'tracks': {'href': 'https://api.spotify.com/v1/users/spotify/playlists/46HbECfImnmmxeQP09Ves4/tracks',
    'total': 0},
   'type': 'playlist',
   'uri': 'spotify:user:spotify:playlist:46HbECfImnmmxeQP09Ves4'},
  {'collaborative': False,
   'external_urls': {'spotify': 'http://o

In [30]:
names = []
total_tracks = []
followers = []
popularity_means = []

playlists = sp.user_playlists('spotify', limit=50, offset=0)

while playlists:
    for i, playlist in enumerate(playlists['items']):
        try:
            metadata = sp.user_playlist('spotify', playlist_id=playlist['id'],fields='followers.total,tracks.items(added_at, track.popularity, track.name)')
            popularities = np.empty(len(metadata['tracks']['items']))
            for index, item in enumerate(metadata['tracks']['items']): 
                popularities[index] = item['track']['popularity']
            popularity_means.append(popularities.mean())
            followers.append(metadata['followers']['total'])
            names.append(playlist['name'])
            total_tracks.append(playlist['tracks']['total'])
            print("%4d %s" % (i + 1 + playlists['offset'], playlist['name']))
        except:
            print("NO METADATA")
    if playlists['next']:
        playlists = sp.next(playlists)
    else:
        playlists = None
        break

//anaconda/lib/python3.5/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)


   1 This Is:  Johnny Cash
   2 This Is: Nina Simone
   3 This Is: Future
   4 Today's Top Hits
   5 Rap Caviar
   6 electroNOW
   7 Afternoon Acoustic
   8 Peaceful Piano
   9 TGIF
  10 The Refugee Playlist
  11 This Is: AC/DC
  12 This Is: Adele
  13 This Is: Al Green
  14 This Is: Aqua
  15 This is: Aretha Franklin
  16 This is: Bee Gees
  17 This Is: blink-182
  18 This is: Bronzeville
  19 This Is: Bruno Mars
  20 This is: Carole King
  21 This Is: Dispatch
  22 This Is: George Michael
  23 This is: Gillian Welch
  24 This Is: James Arthur
  25 This is: Jamiroquai
  26 This Is: John Mayer
  27 This Is: John Legend
  28 This is: Justin Bieber
  29 This Is: Lady Gaga
  30 This Is: LANY
  31 This Is:  Miranda Lambert
  32 This Is: Muse
  33 This is: Neil Young
  34 This Is: New Edition
  35 This Is: One Direction
  36 This Is: OneRepublic
  37 This is: Raffi
  38 This Is: Prince
  39 This Is: Rag'n'Bone Man
  40 This Is: Soundgarden
  41 This Is: Status Quo
  42 This is: Stevie Wonde

In [7]:
#for offset in range(0,1700,50):
#    playlists = sp.user_playlists('spotify', limit=50, offset=offset)
#    while playlists:

### Create a dataframe with the result

In [31]:
playlist_data = pd.DataFrame(
    {'names': names,
     'total_tracks': total_tracks,
     'followers': followers,
     'mean_popularity': popularity_means,
    })

In [35]:
playlist_data.head()

,followers,mean_popularity,names,total_tracks
0,26,NaN,This Is: Johnny Cash,0
1,1684,34.86,This Is: Nina Simone,50
2,30879,63.80,This Is: Future,60
3,13455534,79.60,Today's Top Hits,50
4,5700064,72.42,Rap Caviar,50


### Export the dataframe to a csv for exploration in Tableau

In [33]:
playlist_data.to_csv('playlists.csv')